# Analyzing the logs

In [1]:
# Run from the top of the repository
%cd ..

/Users/pete/calkit/boom-paper


In [2]:
import pandas as pd
import polars as pl
import yaml
import itertools
import os

with open("calkit.yaml") as f:
    calkit_config = yaml.safe_load(f)

# Load the BOOM and Kowalski configurations
boom_params = {}
for i in calkit_config["pipeline"]["stages"]["benchmark-boom"]["iterate_over"]:
    boom_params[i["arg_name"]] = i["values"]

kowalski_params = {}
for i in calkit_config["pipeline"]["stages"]["benchmark-kowalski"][
    "iterate_over"
]:
    kowalski_params[i["arg_name"]] = i["values"]


# Form the matrix of parameters for each
param_names = list(boom_params.keys())
param_vals = list(boom_params.values())
vals_product = list(itertools.product(*param_vals))
results_boom = []

for vals in vals_product:
    current_params = dict(zip(param_names, vals))
    boom_config = (
        f"na={current_params['n_alert_workers']}-"
        f"nml={current_params['n_ml_workers']}-"
        f"nf={current_params['n_filter_workers']}"
    )
    boom_consumer_log_fpath = f"logs/boom-{boom_config}/consumer.log"
    boom_scheduler_log_fpath = f"logs/boom-{boom_config}/scheduler.log"
    # To calculate BOOM wall time, take first timestamp from the consumer log
    # as the start and the last timestamp of the scheduler as the end
    if not os.path.isfile(boom_consumer_log_fpath):
        print(f"WARNING: {boom_consumer_log_fpath} does not exist")
        continue
    with open(boom_consumer_log_fpath) as f:
        line = f.readline()
        t1_b = pd.to_datetime(
            line.split()[2].replace("\x1b[2m", "").replace("\x1b[0m", "")
        )
    with open(boom_scheduler_log_fpath) as f:
        lines = f.readlines()
        line = lines[-3]
        t2_b = pd.to_datetime(
            line.split()[2].replace("\x1b[2m", "").replace("\x1b[0m", "")
        )
    current_params["start_time"] = t1_b
    current_params["end_time"] = t2_b
    boom_wall_time = t2_b - t1_b
    results_boom.append(current_params)

df_boom = pl.DataFrame(results_boom).with_columns(
    (pl.col("end_time") - pl.col("start_time"))
    .dt.total_seconds()
    .alias("wall_time_s")
)

# Kowalski only has one parameter name, so we don't need the product
param_name = list(kowalski_params.keys())[0]
param_vals = kowalski_params[param_name]
results_kowalski = []

for val in param_vals:
    res = {param_name: val}
    kowalski_config = f"n={val}"
    log_fpath = f"logs/kowalski-{kowalski_config}/supervisord.log"
    if not os.path.isfile(log_fpath):
        print(f"WARNING: {log_fpath} does not exist")
        continue
    # To calculate Kowalski wall time, just use the supervisord logs
    with open(log_fpath) as f:
        lines = f.readlines()
        for line in lines:
            if "alert-broker-ztf entered RUNNING state" in line:
                t1_k = pd.to_datetime("T".join(line.split()[:2]))
                break
        for line in lines:
            if "received SIGTERM indicating exit request" in line:
                t2_k = pd.to_datetime("T".join(line.split()[:2]))
                break
    # Print the total time of each, in minutes
    kowalski_wall_time = t2_k - t1_k  # type: ignore
    res["start_time"] = t1_k
    res["end_time"] = t2_k
    results_kowalski.append(res)

df_kowalski = pl.DataFrame(results_kowalski).with_columns(
    (pl.col("end_time") - pl.col("start_time"))
    .dt.total_seconds()
    .alias("wall_time_s")
)

# Write results to file for later analysis
os.makedirs("results", exist_ok=True)
df_boom.write_csv("results/boom.csv")
df_kowalski.write_csv("results/kowalski.csv")

# Calculate throughput factor from a specific config for each
boom_wall_time = df_boom.filter(
    (pl.col("n_alert_workers") == 3)
    & (pl.col("n_ml_workers") == 3)
    & (pl.col("n_filter_workers") == 1)
).select("wall_time_s").row(0)[0]
kowalski_wall_time = (
    df_kowalski.filter(pl.col("n_workers") == 7)
    .select(pl.col("wall_time_s"))
    .row(0)[0]
)

boom_throughput_factor = kowalski_wall_time / boom_wall_time
print(f"BOOM throughput factor: {boom_throughput_factor:.1f}")

BOOM throughput factor: 8.1


In [3]:
pl.Config.set_tbl_rows(50)
df_boom

n_alert_workers,n_ml_workers,n_filter_workers,start_time,end_time,wall_time_s
i64,i64,i64,"datetime[μs, UTC]","datetime[μs, UTC]",i64
1,3,1,2025-08-08 13:19:55.636663 UTC,2025-08-08 13:23:25.574940 UTC,209
1,3,3,2025-08-08 13:24:02.475603 UTC,2025-08-08 13:27:33.568716 UTC,211
1,6,1,2025-08-08 13:28:08.337359 UTC,2025-08-08 13:31:35.476596 UTC,207
1,6,3,2025-08-08 13:32:10.444863 UTC,2025-08-08 13:35:36.482399 UTC,206
1,9,1,2025-08-08 16:31:02.821176 UTC,2025-08-08 16:34:31.773996 UTC,208
1,9,3,2025-08-08 13:40:16.593760 UTC,2025-08-08 13:43:45.830376 UTC,209
3,3,1,2025-08-08 13:44:21.151332 UTC,2025-08-08 13:45:36.562160 UTC,75
3,3,3,2025-08-08 13:46:08.251864 UTC,2025-08-08 13:47:25.196085 UTC,76
3,6,1,2025-08-08 13:47:55.644193 UTC,2025-08-08 13:49:07.049585 UTC,71


In [4]:
df_kowalski

n_workers,start_time,end_time,wall_time_s
i64,datetime[μs],datetime[μs],i64
3,2025-08-08 14:03:53,2025-08-08 14:26:23,1350
6,2025-08-08 14:26:58,2025-08-08 14:38:32,694
7,2025-08-08 15:40:59,2025-08-08 15:51:10,611
9,2025-08-08 14:49:48,2025-08-08 14:57:57,489
12,2025-08-08 14:58:31,2025-08-08 15:05:00,389
15,2025-08-08 15:05:34,2025-08-08 15:10:58,324


In [5]:
# TODO: Put actual results here
kafka_ingest_strate_mbps = "XXXXX"
kafka_ingest_factor = "XXXXX"

template = r"""% This file was automatically generated; edits will be overwritten!
\newcommand{\boomthroughputfactor}{BOOM_THROUGHPUT_FACTOR}
\newcommand{\kakfaingestratembps}{KAFKA_INGEST_STRATE_MBPS}
\newcommand{\kakfaingestfactor}{KAFKA_INGEST_FACTOR}
"""

# This might be better with an f-string, but this is TeX and who wants to deal
# with escaping all the braces?
template = (
    template.replace(
        "BOOM_THROUGHPUT_FACTOR", f"{round(boom_throughput_factor):1d}"
    )
    .replace("KAFKA_INGEST_STRATE_MBPS", kafka_ingest_strate_mbps)
    .replace("KAFKA_INGEST_FACTOR", kafka_ingest_factor)
)

with open("paper/results.tex", "w") as f:
    f.write(template)

In [6]:
# Plot Kowalski scaling
import os

import plotly.graph_objects as go

normalized_throughput = (
    df_kowalski["wall_time_s"].max()
    / df_kowalski["wall_time_s"]
)

fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=df_kowalski["n_workers"],
        y=normalized_throughput,
        mode="markers+lines",
        marker=dict(size=10),
    )
)
fig.update_layout(
    xaxis_title="Number of workers",
    yaxis_title="Normalized throughput",
    title=None,
)
# Shrink top margin
fig.update_layout(margin=dict(t=65))
os.makedirs("paper/figures", exist_ok=True)
fig.write_image("paper/figures/kowalski-scaling.png", width=800, height=600)
fig

In [7]:
# Plot BOOM scalability
import os

import plotly.graph_objects as go

normalized_throughput = (
    df_kowalski["wall_time_s"].max()
    / df_kowalski["wall_time_s"]
)

fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=df_kowalski["n_workers"],
        y=normalized_throughput,
        mode="markers+lines",
        marker=dict(size=10),
    )
)
fig.update_layout(
    xaxis_title="Number of workers",
    yaxis_title="Normalized throughput",
    title=None,
)
# Shrink top margin
fig.update_layout(margin=dict(t=65))
os.makedirs("paper/figures", exist_ok=True)
fig.write_image("paper/figures/kowalski-scaling.png", width=800, height=600)
fig